In [3]:
!pip install nibabel

In [30]:
import matplotlib.pylab as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import nibabel as nib
from models import unet
from utils import min_max_normalization

In [31]:
def dice_coef(y_true, y_pred):
    ''' Metric used for CNN training'''
    smooth = 1.0 #CNN dice coefficient smooth
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    ''' Loss function'''
    return -(dice_coef(y_true, y_pred)) # try negative log of the DICE* (-tf.math.log())

In [32]:
def pad_volume(data,multiple):
    _,d2,d3 = data.shape
    rem2 = multiple - d2%multiple 
    rem3 = multiple - d3%multiple
    return np.pad(data,pad_width = ((0,0),(0,rem2),(0,rem3)))

In [33]:
data_path = "C:\\Users\\Omega\\Documents\\School\\Winter 2022\\enel645-final-project\\unet-segmentation\\Data\\Original"
test_files = "C:\\Users\\Omega\\Documents\\School\\Winter 2022\\enel645-final-project\\unet-segmentation\\Data\\test-set.txt"
test_files = np.loadtxt(test_files,dtype = str)
test_files = [os.path.join(data_path, f + ".gz") for f in test_files]

label01_path = "C:\\Users\\Omega\\Documents\\School\\Winter 2022\\enel645-final-project\\unet-segmentation\\Data\\Brain-masks"
label01_files = "C:\\Users\\Omega\\Documents\\School\\Winter 2022\\enel645-final-project\\unet-segmentation\\Data\\test-set.txt"
label01_files = np.loadtxt(label01_files,dtype = str)
label01_files = [os.path.join(label01_path, f.split('.')[0] + "_staple.nii.gz") for f in label01_files]

label02_path = "C:\\Users\\Omega\\Documents\\School\\Winter 2022\\enel645-final-project\\unet-segmentation\\Data\\WM-GM-CSF"
label02_files = "C:\\Users\\Omega\\Documents\\School\\Winter 2022\\enel645-final-project\\unet-segmentation\\Data\\test-set.txt"
label02_files = np.loadtxt(label02_files,dtype = str)
label02_files = [os.path.join(label02_path, f + ".gz") for f in label02_files]

files = zip(test_files, label01_files, label02_files)

multiple = 2**4 # the network has 4 max-pooling layers

In [34]:
model_name = "unet_multitask.h5"
model = unet(input_shape=(None, None, 1))
model.load_weights(model_name)
model.compile(loss = dice_coef_loss, optimizer=Adam())

In [43]:
image = nib.load(label01_files[0]).get_fdata()
image.shape

(224, 256, 256)

In [48]:
idx = 120
scores01 = np.zeros(48)
scores02 = np.zeros(48)

for i, (ii, jj, kk) in enumerate(zip(test_files, label01_files, label02_files)):
    data = nib.load(ii).get_fdata()
    label01 = nib.load(jj).get_fdata()
    label02 = nib.load(kk).get_fdata()
    
    data = min_max_normalization(data)
    data = data.transpose(2,0,1) # axial slices first
    label01 = label01.transpose(2,0,1) # axial slices first
    label02 = label02.transpose(2,0,1) # axial slices first
    
    
    H,W,Z = data.shape
    data_padded = pad_volume(data,16)
    pred01,pred02 = model.predict(data_padded[:,:,:,np.newaxis])
    pred01 = pred01[:,:W,:Z,0] > 0.5
    pred01 = pred01.astype(np.double)
    pred02 = pred02[:,:W,:Z,:].argmax(axis = -1)*pred01
    
    # print("Finished loop: " + str(i))
    scores01[i] = dice_coef_loss(label01, pred01)
    scores02[i] = dice_coef_loss(label02, pred02)
    print("Finished loop: " + str(i))
    print("Score 01: " + str(scores01[i]))
    print("Score 02: " + str(scores01[i]))
    
    # print(score01)
    
    # score01 = model.evaluate(data_padded[:,:,:,np.newaxis], 
    # score02 = model.evaluate(data_padded[:,:,:,np.newaxis], 
    # plt.figure()
    # plt.imshow(data[idx], cmap = "gray")
    # plt.imshow(pred01[idx], alpha = 0.5)
    # plt.show()
    # plt.figure()
    # plt.imshow(data[idx], cmap = "gray")
    # plt.imshow(pred02[idx], alpha = 0.5)
    # plt.show()    

NameError: name 'scores01' is not defined

In [ ]:
with open('scores01.npy', 'wb') as f:
    np.save(f, scores01)
with open('scores02.npy', 'wb') as f:
    np.save(f, scores02)